In [1]:
%pylab inline
import pandas as pd
import text_util as textutil
import tensorflow as tf
from sklearn.model_selection import train_test_split
from gensim.models.word2vec import Word2Vec

Populating the interactive namespace from numpy and matplotlib


D:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
#载入数据
pos = textutil.load_txt("pos.txt")
neg = textutil.load_txt("neg.txt")

In [3]:
#分词，处理数据
pos = textutil.seg_words_with_blank(pos)
neg = textutil.seg_words_with_blank(neg)

#混合数据
randIt = []
data = []
labels = []
for i in range(len(pos)):
    randIt.append([pos[i],[0,1]]) #正面
for i in range(len(neg)):
    randIt.append([neg[i],[1,0]]) #负面
for i in range(len(randIt)):
    data.append(randIt[i][0])
    labels.append(randIt[i][1])

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\lenovo\AppData\Local\Temp\jieba.cache
Loading model cost 1.070 seconds.
Prefix dict has been built succesfully.


In [4]:
data[1057]

'二手 东西 无语 无意 中 提醒 事项 无语 死 太 不好'

In [5]:
labels[1057]

[1, 0]

In [6]:
#调整超参数的设置
batch_size = 50 #样本数
lstm_size = 128 #lstm cell里面的神经元数目
num_layers = 3#堆叠
target_classes = 2#最终分为2类
learning_rate = 0.001
keep_prob = 0.5
num_keywords = 2000
word_classes = 100 #词向量维数为100维

In [7]:
#提取关键字
keywords = textutil.key_words(data,num_keywords)

In [8]:
#创建由关键字组成的稀疏矩阵
textmatrix = textutil.count_sparse_matrix(data,True)
textmatrix = textmatrix[keywords]

In [9]:
textmatrix.head(5)

,买,不错,京东,降价,东西,说,客服,差评,屏幕,价格,...,一问,感太强,尾款,斤斤计较,不贵,预装,比上,提高,接到,当作
0,0,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
#整理数据
wordlists, steps = textutil.matrix_to_wordlists(textmatrix)
worddata = pd.DataFrame({"WORD": wordlists, "STEP":steps})

In [11]:
worddata.head(5)

,STEP,WORD
0,23,"[京东, 价格, 速度, ipad, 物流, 平板, 满意, 几天, 购买, 下单, 好用,..."
1,29,"[买, 东西, 正品, 苹果, ipad, 真的, 第一次, 发现, 找, 会员, 建议, ..."
2,42,"[京东, 客服, 正品, 苹果, ipad, 激活, 平板, 第一次, 问, 发货, 消费者..."
3,23,"[买, 说, 屏幕, 速度, 物流, 几天, 真的, 下单, 评价, 清晰, 买回来, 网上..."
4,25,"[买, 不错, 速度, 激活, 平板, 购买, 第一次, 很快, 运行, 系统, 中, 不卡..."


In [12]:
#拆分训练集和数据集
traindata,testdata,trainlabel,testlabel = train_test_split(worddata, labels, test_size = 0.2)

In [13]:
#载入词向量model
model = Word2Vec.load("评论.model")

In [14]:
#词向量读取方法
def word_to_vec(word):
    try:
        wordvec = model[word]
    except KeyError as e:
        print(word, "不存在词向量")
        wordvec = np.array([0] * 100)
    return wordvec

In [15]:
#内含one-hot的get-batches
def get_batches(traindata,trainlabel,batch_size, word_classes):
    n_batches = len(traindata) // batch_size
    traindata = traindata.iloc[ : batch_size * n_batches, :]

    for i in range(n_batches):
        dataframe_x = traindata.iloc[i * batch_size : (i + 1) * batch_size]
        batch_y = np.array(trainlabel[i * batch_size : (i + 1) * batch_size])
        step = max(dataframe_x["STEP"])
        word_x = dataframe_x["WORD"]
        batch_item = []
        for item in word_x:
            steps_x = []
            for j in range(step):
                if j < len(item):
                    step_word = word_to_vec(item[j])
                else:
                    step_word = [0]*word_classes #补0
                steps_x.append(step_word)
            batch_item.append(np.array(steps_x))
        batch_x = np.array(batch_item)
        yield batch_x, batch_y, step

In [16]:
#get_batches方法的测试
i = 1
for x,y,step in get_batches(traindata, trainlabel, batch_size, word_classes):
    print("*************第",i,"次**********************")
    print("step:", step)
    print("x的shape:", x.shape)
    print("y的shape:",y.shape)
    i += 1
    print("********************************************")

D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  after removing the cwd from sys.path.


*************第 1 次**********************
step: 24
x的shape: (50, 24, 100)
y的shape: (50, 2)
********************************************
*************第 2 次**********************
step: 52
x的shape: (50, 52, 100)
y的shape: (50, 2)
********************************************
*************第 3 次**********************
step: 32
x的shape: (50, 32, 100)
y的shape: (50, 2)
********************************************
*************第 4 次**********************
step: 34
x的shape: (50, 34, 100)
y的shape: (50, 2)
********************************************
*************第 5 次**********************
step: 55
x的shape: (50, 55, 100)
y的shape: (50, 2)
********************************************
*************第 6 次**********************
step: 28
x的shape: (50, 28, 100)
y的shape: (50, 2)
********************************************
*************第 7 次**********************
step: 28
x的shape: (50, 28, 100)
y的shape: (50, 2)
********************************************
*************第 8 次**********************
step: 31
x的sha

In [17]:
#设置输入placeholder
def build_inputs(batch_size, word_classes, target_classes):
    inputs = tf.placeholder(tf.float32, [batch_size, None, word_classes], name = "inputs")
    targets = tf.placeholder(tf.float32, [batch_size,target_classes], name = "targets")
    
    keep_prob = tf.placeholder(tf.float32, name = "keep_prob")
    return inputs, targets, keep_prob

In [18]:
#构建lstm单元
def lstm_cell(lstm_size, keep_prob):
    cell = tf.contrib.rnn.LSTMCell(lstm_size, reuse=tf.get_variable_scope().reuse)
    return tf.contrib.rnn.DropoutWrapper(cell, output_keep_prob=keep_prob)

#构建多层lstm
def build_lstm(lstm_size, num_layers, batch_size, keep_prob):
    cell = tf.contrib.rnn.MultiRNNCell([lstm_cell(lstm_size,keep_prob) for _ in range(num_layers)],
                                       state_is_tuple=True)
    #初始化所有权重为0
    initial_state = cell.zero_state(batch_size, tf.float32)
    return cell, initial_state

In [31]:
#构建返回结果 包含两层全连接层
def build_output(lstm_output, lstm_size, target_classes):
    seq_output = tf.concat(lstm_output, axis=1)
    seq_output = seq_output[:, -1, :] #选取最后一个值输出
    print('lstm输出到全连接层的结构:',seq_output)
    x = tf.reshape(seq_output, [-1, lstm_size])
    print("reshape后的结构:", x)
    
    with tf.variable_scope("softmax"):
        hidden_w = tf.Variable(tf.truncated_normal((lstm_size, lstm_size // 2), stddev=0.1))
        hidden_b = tf.Variable(tf.zeros(lstm_size // 2))
        
        softmax_w = tf.Variable(tf.truncated_normal((lstm_size // 2, target_classes), stddev=0.1))
        softmax_b = tf.Variable(tf.zeros(target_classes))
        
    hidden = tf.matmul(x, hidden_w) + hidden_b
    hidden = tf.nn.relu(hidden)
        
    logist = tf.matmul(hidden, softmax_w) + softmax_b
    out = tf.nn.softmax(logist, name = "predictions")
    print("out的结构:",out)
    print("logist的结构：",logist)
    return out,logist

In [20]:
#构建损失 和 准确率
def build_loss(logits, targets):
    loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=targets)
    loss = tf.reduce_mean(loss)
    return loss

In [21]:
#准确率
def build_accuracy(logits, targets):
    correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(targets, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
    return accuracy

In [22]:
#构建梯度下降optimizer
def build_optimizer(loss, learning_rate, grad_clip):
    tvars = tf.trainable_variables()
    grads,_ = tf.clip_by_global_norm(tf.gradients(loss, tvars), grad_clip)
    train_op = tf.train.AdamOptimizer(learning_rate)
    optimizer = train_op.apply_gradients(zip(grads, tvars))
    return optimizer

In [23]:
class CharRNN:
    def __init__(self, word_classes = word_classes, target_classes = target_classes, 
                 batch_size = 100,lstm_size = 20, num_layers = 5, learning_rate = 0.01,
                 grad_clip = 5):
        tf.reset_default_graph()
        self.inputs, self.targets, self.keep_prob = build_inputs(batch_size,word_classes, target_classes)
        print("输入的结构为：",self.inputs)
        
        #创建lstm网络
        cell, self.initial_state = build_lstm(lstm_size, num_layers, 
                                              batch_size, self.keep_prob)

        outputs, state = tf.nn.dynamic_rnn(cell, self.inputs, 
                                         initial_state = self.initial_state)
        
        self.final_state = state
        self.prediction, self.logits = build_output(outputs, lstm_size,target_classes)
        self.loss = build_loss(self.logits, self.targets)
        self.accuracy = build_accuracy(self.logits, self.targets)
        self.optimizer = build_optimizer(self.loss, learning_rate, grad_clip)

In [ ]:
#开始训练
epochs = 10
model_train = CharRNN(word_classes=word_classes, target_classes=target_classes,batch_size = batch_size, 
               lstm_size = lstm_size, num_layers = num_layers,learning_rate = learning_rate)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    print("\n************************训练开始**************************")
    count = 0
    while True:
        for i in range(epochs):
            new_state = sess.run(model_train.initial_state)
            loss = 0
            for x, y, step in get_batches(traindata, trainlabel, batch_size, word_classes):
                feed ={
                    model_train.inputs : x,
                    model_train.targets : y,
                    model_train.keep_prob : keep_prob,
                    model_train.initial_state:new_state}
                batch_loss,_ = sess.run([model_train.loss,model_train.optimizer],feed_dict = feed)
                loss += batch_loss
            print("第",(epochs * count + i + 1), "轮,损失为：",loss)
        count += 1
    
        #预测前100个
        test_new_state = sess.run(model_train.initial_state)
        accuracy = 0
        for x,y,step in get_batches(testdata, testlabel, batch_size, word_classes):
            feed = {
                model_train.inputs : x,
                model_train.targets : y,
                model_train.keep_prob : 1.,
                model_train.initial_state:test_new_state}
            test_accuracy = sess.run(model_train.accuracy, feed_dict = feed)
            accuracy += test_accuracy
        print("测试集的预测准确率为:", accuracy / (testdata.shape[0] // batch_size))
        if test_accuracy > 0.90: 
            print("\n************************训练结束**************************")
            break

输入的结构为： Tensor("inputs:0", shape=(50, ?, 100), dtype=float32)
lstm输出到全连接层的结构: Tensor("strided_slice:0", shape=(50, 128), dtype=float32)
reshape后的结构: Tensor("Reshape:0", shape=(50, 128), dtype=float32)
out的结构: Tensor("predictions:0", shape=(50, 2), dtype=float32)
logist的结构： Tensor("add_1:0", shape=(50, 2), dtype=float32)
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.


************************训练开始**************************


D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  after removing the cwd from sys.path.


第 1 轮,损失为： 22.909049406647682
第 2 轮,损失为： 16.40893915295601
第 3 轮,损失为： 15.72669368982315
第 4 轮,损失为： 15.316183179616928
第 5 轮,损失为： 13.735258154571056
第 6 轮,损失为： 13.5099418759346
第 7 轮,损失为： 13.544471211731434
第 8 轮,损失为： 12.999535702168941
第 9 轮,损失为： 13.30509139597416
第 10 轮,损失为： 12.861951164901257
测试集的预测准确率为: 0.8583333293596903
第 11 轮,损失为： 12.649923138320446
第 12 轮,损失为： 13.219664424657822
第 13 轮,损失为： 12.559312030673027
第 14 轮,损失为： 12.316813163459301
第 15 轮,损失为： 11.854339078068733
第 16 轮,损失为： 11.814427096396685
第 17 轮,损失为： 12.237268686294556
第 18 轮,损失为： 11.699782200157642
第 19 轮,损失为： 11.495755344629288
第 20 轮,损失为： 11.477240107953548
测试集的预测准确率为: 0.8783333251873652
第 21 轮,损失为： 11.25829280167818
第 22 轮,损失为： 11.308643288910389
第 23 轮,损失为： 11.558814622461796
第 24 轮,损失为： 11.804284676909447
第 25 轮,损失为： 11.46411182358861
第 26 轮,损失为： 11.420957949012518
第 27 轮,损失为： 11.509524315595627
第 28 轮,损失为： 11.32996991276741
第 29 轮,损失为： 11.092539053410292
第 30 轮,损失为： 11.211321409791708
测试集的预测准确率为: 0.88499999543

第 243 轮,损失为： 1.7744455146748805
第 244 轮,损失为： 1.7096681341936346
第 245 轮,损失为： 1.351924834860256
第 246 轮,损失为： 0.7757091304520145
第 247 轮,损失为： 0.86824489798164
第 248 轮,损失为： 0.9116920511587523
第 249 轮,损失为： 1.5050885889213532
第 250 轮,损失为： 2.4595063200686127
测试集的预测准确率为: 0.8566666593154272
第 251 轮,损失为： 1.6922422838397324
第 252 轮,损失为： 2.0363009164575487
第 253 轮,损失为： 1.632091503648553
第 254 轮,损失为： 1.4238127495045774
第 255 轮,损失为： 1.3137337287771516
第 256 轮,损失为： 0.9516651860321872
第 257 轮,损失为： 0.8045245542452903
第 258 轮,损失为： 0.6451569828859647
第 259 轮,损失为： 1.7136495485319756
第 260 轮,损失为： 1.0259902767138556
测试集的预测准确率为: 0.8633333245913187
第 261 轮,损失为： 0.6927971277327742
第 262 轮,损失为： 0.5959971375705209
第 263 轮,损失为： 1.2732655411455198
第 264 轮,损失为： 1.4011616632924415
第 265 轮,损失为： 1.2074377862154506
第 266 轮,损失为： 1.1285119100939482
第 267 轮,损失为： 1.8886602014536038
第 268 轮,损失为： 1.144637111457996
第 269 轮,损失为： 0.786472833249718
第 270 轮,损失为： 0.6896210941922618
测试集的预测准确率为: 0.8599999944368998
第 271 轮,损失为： 2.09

第 479 轮,损失为： 0.8379889644493232
第 480 轮,损失为： 0.5163989817519905
测试集的预测准确率为: 0.8683333347241083
第 481 轮,损失为： 0.41389543076365953
第 482 轮,损失为： 0.41381498042755993
第 483 轮,损失为： 0.5125828895306768
第 484 轮,损失为： 0.5431211327850178
第 485 轮,损失为： 0.4285311785170052
第 486 轮,损失为： 0.4096752705536346
第 487 轮,损失为： 0.3697954735825988
第 488 轮,损失为： 0.40828990418231115
第 489 轮,损失为： 0.9893583820157801
第 490 轮,损失为： 1.1793392001600296
测试集的预测准确率为: 0.8666666646798452
第 491 轮,损失为： 0.9989876629115315
第 492 轮,损失为： 3.4230001309188083
第 493 轮,损失为： 1.58552852680441
第 494 轮,损失为： 1.7923072298290208
第 495 轮,损失为： 0.7496557378326543
第 496 轮,损失为： 0.6949829531658906
第 497 轮,损失为： 0.998632898874348
第 498 轮,损失为： 1.296364915004233
第 499 轮,损失为： 0.7827765690453816
第 500 轮,损失为： 0.575384886644315
测试集的预测准确率为: 0.8683333247900009
第 501 轮,损失为： 0.40756200495525263
第 502 轮,损失为： 0.37004094207804883
第 503 轮,损失为： 0.45816403861317667
第 504 轮,损失为： 0.5357357586399303
第 505 轮,损失为： 0.5432224419855629
第 506 轮,损失为： 0.4861968639488623
第 507 轮,损失

第 712 轮,损失为： 0.33276985132988557
第 713 轮,损失为： 1.7578116886675161
第 714 轮,损失为： 2.2387119559571147
第 715 轮,损失为： 2.161346765817143
第 716 轮,损失为： 1.1671518919611117
第 717 轮,损失为： 1.1147308362269541
第 718 轮,损失为： 0.7713624360330869
第 719 轮,损失为： 0.6152400653372752
第 720 轮,损失为： 0.4260268656325934
测试集的预测准确率为: 0.8716666599114736
第 721 轮,损失为： 0.3448669091503689
第 722 轮,损失为： 0.6940329500048392
第 723 轮,损失为： 0.979322266915915
第 724 轮,损失为： 0.9669055569174816
第 725 轮,损失为： 1.1703324613627046
第 726 轮,损失为： 0.8420206212176709
第 727 轮,损失为： 0.7597638149454724
第 728 轮,损失为： 0.5134799802544876
第 729 轮,损失为： 0.4323004544039577
第 730 轮,损失为： 0.4174335492443788
测试集的预测准确率为: 0.8816666603088379
第 731 轮,损失为： 0.40582196297509654
第 732 轮,损失为： 0.33063323484566354
第 733 轮,损失为： 0.35130302539164404
第 734 轮,损失为： 0.3847121965111455
第 735 轮,损失为： 0.43465152542989927
第 736 轮,损失为： 0.33146839984055987
第 737 轮,损失为： 0.33666795578437814
第 738 轮,损失为： 0.5267466433415393
第 739 轮,损失为： 1.101333500842884
第 740 轮,损失为： 1.8413863148307428
测试集的预测

第 945 轮,损失为： 0.32460950737549865
第 946 轮,损失为： 0.27318484153784084
第 947 轮,损失为： 0.3077986434475406
第 948 轮,损失为： 0.29269222968611075
第 949 轮,损失为： 0.348884059260115
第 950 轮,损失为： 0.5081710677275169
测试集的预测准确率为: 0.8733333249886831
第 951 轮,损失为： 0.5015053777792673
第 952 轮,损失为： 0.518443076784024
第 953 轮,损失为： 0.39493730875256006
第 954 轮,损失为： 0.35460534997764626
第 955 轮,损失为： 0.3074008710882481
第 956 轮,损失为： 0.37509177466745314
第 957 轮,损失为： 0.4452090273575777
第 958 轮,损失为： 0.644696770936207
第 959 轮,损失为： 0.43861087158984446
第 960 轮,损失为： 0.31103885247739527
测试集的预测准确率为: 0.8749999950329462
第 961 轮,损失为： 0.3909960854966812
第 962 轮,损失为： 0.3933375573797093
第 963 轮,损失为： 0.6975221783750385
第 964 轮,损失为： 0.8325748392817331
第 965 轮,损失为： 0.7874466350185685
第 966 轮,损失为： 0.6788692104601068
第 967 轮,损失为： 1.1883166464831447
第 968 轮,损失为： 0.9563414790245588
第 969 轮,损失为： 0.8973426312149968
第 970 轮,损失为： 0.5291441013978329
测试集的预测准确率为: 0.878333330154419
第 971 轮,损失为： 0.8528503851412097
第 972 轮,损失为： 0.5999707071168814
第 973 轮

第 1172 轮,损失为： 0.39592749181974796
第 1173 轮,损失为： 0.637680380117672
第 1174 轮,损失为： 0.4711634547675203
第 1175 轮,损失为： 0.4109571994449652
第 1176 轮,损失为： 0.48009292877759435
第 1177 轮,损失为： 0.5389695931626193
第 1178 轮,损失为： 0.37324922077095835
第 1179 轮,损失为： 0.27373265028472815
第 1180 轮,损失为： 0.2636280215538136
测试集的预测准确率为: 0.8699999948342642
